In [1]:
import dgl
import pandas as pd
import networkx as nx
import itertools
from itertools import combinations
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import Configuration

In [2]:
# Define the message and reduce function
class GCNLayer(nn.Module):
    def gcn_message(self, edges):
        return {'msg' : edges.src['h']}

    def gcn_reduce(self, nodes):
        return {'h' : torch.sum(nodes.mailbox['msg'], dim=1)}

    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, inputs):
        g.ndata['h'] = inputs
        g.send(g.edges(), self.gcn_message)
        g.recv(g.nodes(), self.gcn_reduce)
        h = g.ndata.pop('h')
        return self.linear(h)

In [3]:
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_feats, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.gcn1(g, inputs)
        h = torch.relu(h)
        h = self.gcn2(g, h)
        return h

In [4]:
if __name__ == "__main__":
    df = pd.read_csv(Configuration.PROCESSED_FILE_PATH)
    net = GCN(df.shape[1], 10, 2)